In [3]:
from datasets import load_dataset

dataset = load_dataset("EleutherAI/pythia-memorized-evals")

from pandarallel import pandarallel
import multiprocessing
pandarallel.initialize(progress_bar=True, nb_workers=multiprocessing.cpu_count())

import pickle

Found cached dataset parquet (/home/mchorse/.cache/huggingface/datasets/EleutherAI___parquet/EleutherAI--pythia-memorized-evals-623aaa371a33821a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 35.78it/s]


INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
from transformers import AutoTokenizer

names = [
    'pythia-70m',
    'pythia-160m',
    'pythia-410m',
    'pythia-1b',
    'pythia-1.4b',
    'pythia-2.8b',
    'pythia-6.9b'
]

sizes = []

for name in names:
    sizes.append(name.split("-")[-1])

deduped_names = []
for name in names:
    deduped_names.append(name + '-deduped')

names = names + deduped_names

In [7]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.1/198.1 kB 36.8 MB/s eta 0:00:00


In [9]:
names, sizes

(['pythia-70m',
  'pythia-160m',
  'pythia-410m',
  'pythia-1b',
  'pythia-1.4b',
  'pythia-2.8b',
  'pythia-6.9b',
  'pythia-70m-deduped',
  'pythia-160m-deduped',
  'pythia-410m-deduped',
  'pythia-1b-deduped',
  'pythia-1.4b-deduped',
  'pythia-2.8b-deduped',
  'pythia-6.9b-deduped'],
 ['70m', '160m', '410m', '1b', '1.4b', '2.8b', '6.9b'])

In [10]:
dataset_split = []
def detokenize_list_of_tokens(tokens, tokenizer):
    toks = []
    for tok in tokens:
        toks.append(tokenizer.decode(tok))
    return toks

for name in names:
    curr_name = f"EleutherAI/{name}"
    tokenizer = AutoTokenizer.from_pretrained(
        curr_name,
    )
    subset_name = ""
    if 'deduped' in curr_name:
      subset_name += 'deduped.'
    else:
      subset_name += 'duped.'
    size = name.split("-")[1]
    subset_name += size
    dataset_split.append(subset_name)
    df = dataset[subset_name].to_pandas()
    df['detokenized'] = df['tokens'].parallel_apply(detokenize_list_of_tokens, args=(tokenizer,))
    pickle.dump(df, open(f"./data/{name}.pkl", "wb"))
    # try to load
    df = pickle.load(open(f"./data/{name}.pkl", "rb"))